## All Config

In [9]:
import json
import os
import re
from collections import defaultdict

# === SETTINGS ===
pbip_file_name = 'makingVisualThemes'  # base name of PBIP project
export_only_default_selectors = False  # Set True to include only items with selector.id = 'default' or missing

# === PATHS ===
report_path = os.path.join(f"{pbip_file_name}.Report", "report.json")
# Renamed the output directory for visual configs
output_dir = f"{pbip_file_name}_visual_configs"
os.makedirs(output_dir, exist_ok=True) # Ensure the visual configs directory exists

# === STATIC THEME HEADER (NEW) ===
static_theme_header = {
  "$schema": "Reference reportThemeSchema-2.143 (May 2025).json",
  "name": "Achmad Farizky Custom 3.0",
  "dataColors": [
    "#00668c",
    "#71c4ef",
    "#d4eaf7",
    "#b6ccd8",
    "#313d44",
    "#3b3c3d",
    "#308C9A",
    "#C05B5B"
  ],
  "good": "#308C9A",
  "bad": "#C05B5B",
  "neutral": "#BCBBBF",
  "maximum": "#326696",
  "minimum": "#B34E4E",
  "center": "#D8DBDF",
  "background": "#FFFFFF",
  "backgroundLight": "#F3F2F1",
  "backgroundNeutral": "#edf1f4",
  "foreground": "#243140",
  "foregroundNeutralSecondary": "#605E5C",
  "foregroundNeutralTertiary": "#B3B0AD",
  "firstLevelElements": "#30465F",
  "secondLevelElements": "#455d79",
  "tableAccent": "#e7ecf0",
  "visualStyles": {} # This will be populated by the script's output
}


# === VALUE EXTRACTION UTILITY ===
def extract_value(value):
    if isinstance(value, dict):
        if "expr" in value:
            expr = value["expr"]
            if isinstance(expr, dict) and "Literal" in expr:
                literal = expr["Literal"]
                if isinstance(literal, dict) and "Value" in literal:
                    val = literal["Value"]
                    if isinstance(val, str):
                        val = val.strip("'")  # remove surrounding quotes
                        val_lower = val.lower()
                        if val_lower == "true":
                            return True
                        elif val_lower == "false":
                            return False
                        # Numeric literal with suffix
                        if val.endswith("D") or val.endswith("L"):
                            num_str = val[:-1]
                            try:
                                return int(num_str)
                            except ValueError:
                                try:
                                    return float(num_str)
                                except ValueError:
                                    return val
                        try:
                            if '.' in val:
                                return float(val)
                            return int(val)
                        except ValueError:
                            return val
                    return val
        # Recursively process nested dictionaries
        return {k: extract_value(v) for k, v in value.items()}

    elif isinstance(value, list):
        return [extract_value(item) for item in value]

    return value

# === HELPER FUNCTION TO PROCESS OBJECTS ===
def process_objects_for_theme(objects_data, export_default_only):
    processed_output = {}
    if not objects_data:
        return processed_output

    for key, items in objects_data.items():
        processed_output[key] = []
        for item in items:
            selector = item.get("selector")
            selector_id = None
            if selector is not None:
                selector_id = selector.get("id")

            if export_default_only:
                if selector is not None:
                    if selector_id is None or selector_id != "default":
                        continue
            
            flat_properties = {
                prop_key: extract_value(prop_val)
                for prop_key, prop_val in item.get("properties", {}).items()
            }
            if selector_id is not None:
                flat_properties["$id"] = selector_id

            if flat_properties not in processed_output[key]:
                processed_output[key].append(flat_properties)
    
    return {k: v for k, v in processed_output.items() if v}

# === MAIN PROCESSING ===
with open(report_path, "r", encoding="utf-8") as f:
    data = json.load(f)

for section_index, section in enumerate(data.get("sections", [])):
    page_name = section.get("displayName", f"Page_{section_index}")
    visual_containers = section.get("visualContainers", [])

    page_theme_output = defaultdict(dict)
    visual_type_count = defaultdict(int)

    for vc_index, vc in enumerate(visual_containers):
        config_text = vc.get("config")
        if not config_text:
            continue

        try:
            config = json.loads(config_text)
        except json.JSONDecodeError:
            continue

        single_visual = config.get("singleVisual")
        if not single_visual:
            continue

        visual_type = single_visual.get("visualType", "UnknownType")
        
        # Process vcObjects and objects
        vc_objects_data = single_visual.get("vcObjects")
        objects_data = single_visual.get("objects")

        themed_vc_objects = process_objects_for_theme(vc_objects_data, export_only_default_selectors)
        
        themed_objects = {}
        if objects_data:
            for key, items in objects_data.items():
                if key == "general":
                    continue
                processed_items = []
                for item in items:
                    selector = item.get("selector")
                    selector_id = None
                    if selector is not None:
                        selector_id = selector.get("id")

                    if export_only_default_selectors:
                        if selector is not None:
                            if selector_id is None or selector_id != "default":
                                continue
                    
                    flat_properties = {
                        prop_key: extract_value(prop_val)
                        for prop_key, prop_val in item.get("properties", {}).items()
                    }
                    if selector_id is not None:
                        flat_properties["$id"] = selector_id

                    if flat_properties not in processed_items:
                        processed_items.append(flat_properties)
                
                if processed_items:
                    themed_objects[key] = processed_items
        
        # Conflict Resolution: vcObjects priority
        combined_themed_output = dict(themed_vc_objects) 
        for key, value in themed_objects.items():
            if key not in combined_themed_output:
                combined_themed_output[key] = value

        combined_themed_output = {k: v for k, v in combined_themed_output.items() if v}
        
        # === FILE NAME FOR RAW CONFIG ===
        safe_page_name = re.sub(r'[^\w\-]+', '_', page_name)
        safe_visual_type = re.sub(r'[^\w\-]+', '_', visual_type)
        config_filename = f"config_{safe_page_name}_{vc_index}_{safe_visual_type}.json"
        config_path = os.path.join(output_dir, config_filename)

        # Combine vc_objects_data and objects_data for raw dump
        raw_dump_content = {}
        if vc_objects_data:
            raw_dump_content["vcObject"] = vc_objects_data
        if objects_data:
            raw_dump_content["Object"] = objects_data
        
        if raw_dump_content:
            with open(config_path, "w", encoding="utf-8") as config_file:
                json.dump(raw_dump_content, config_file, indent=2)

        visual_type_key = visual_type
        visual_type_count[visual_type] += 1
        if visual_type_count[visual_type] > 1:
            visual_type_key = f"{visual_type}___{visual_type_count[visual_type]}"

        if combined_themed_output:
            page_theme_output["visualStyles"][visual_type_key] = {"*": combined_themed_output}

    # === PAGE CONFIG HANDLING ===
    page_config_text = section.get("config")
    if page_config_text:
        try:
            page_config = json.loads(page_config_text)
            page_config_output = process_objects_for_theme(
                page_config.get("objects", {}), export_only_default_selectors
            )
            
            # Save page_config raw
            page_config_filename = f"config_{safe_page_name}.json"
            page_config_path = os.path.join(output_dir, page_config_filename)
            with open(page_config_path, "w", encoding="utf-8") as f:
                json.dump(page_config, f, indent=2)

            if page_config_output:
                page_theme_output["visualStyles"]["*"] = {"*": page_config_output}
        except json.JSONDecodeError:
            print(f"Warning: Skipping invalid page config for {page_name}")

# === SAVE ONE THEME FILE PER PAGE ===
    ordered_visual_styles = {}
    vs = page_theme_output.get("visualStyles", {})
    if "*" in vs:
        ordered_visual_styles["*"] = vs.pop("*")
    for key in sorted(vs):
        ordered_visual_styles[key] = vs[key]

    # Create the final theme output by merging the static header and generated visualStyles
    final_theme_output = static_theme_header.copy() # Start with a copy of the static header
    final_theme_output["visualStyles"] = ordered_visual_styles # Assign the generated visualStyles

    # Theme filename format: <pbip filename>_theme_<page name>.json
    theme_filename = f"{pbip_file_name}_theme_{safe_page_name}.json"
    theme_path = theme_filename # Still saved in the current directory
    
    if final_theme_output.get("visualStyles"): # Check if visualStyles has content
        with open(theme_path, "w", encoding="utf-8") as theme_file:
            json.dump(final_theme_output, theme_file, indent=2)
        print(f"Saved: {theme_filename} with all themes for page '{page_name}' to the current directory.")
    else:
        # If visualStyles is empty but other static data exists, you might still want to save it.
        # This condition now only checks visualStyles to avoid saving an entirely empty file,
        # but the static header will always provide some content.
        # For a truly empty scenario, you might check if 'final_theme_output' is just the static header.
        if final_theme_output != static_theme_header: # Only save if something beyond the base header was added
            with open(theme_path, "w", encoding="utf-8") as theme_file:
                json.dump(final_theme_output, theme_file, indent=2)
            print(f"Saved: {theme_filename} with base themes for page '{page_name}' to the current directory.")
        else:
            print(f"Skipped saving empty theme for page '{page_name}' (only static header would be present).")

Saved: makingVisualThemes_theme_page1.json with all themes for page 'page1' to the current directory.
